#1 Introducción

El siguiente cuaderno realiza la busqueda de un número flotante utilizando paralelismo usando la CPU Y la GPU.

El objetivo es comparar los tiempos de ejecución de ambas versiones del algoritmo de busqueda.

#2 Armado del ambiente

Instala en el cuaderno el módulo CUDA de Python.

In [2]:
!pip install pycuda

     |████████████████████████████████| 1.6MB 21.1MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 81kB 12.2MB/s 
  Created wheel for pycuda: filename=pycuda-2020.1-cp36-cp36m-linux_x86_64.whl size=621228 sha256=717177d2367f9ec2bc93a05f0aa6871b6c5ee9cc399e77b178a32b939b5417c8
  Stored in directory: /root/.cache/pip/wheels/8f/78/d1/5bb826f81d9d490297a348d818ff3ee6dd6f2075b06dde6ea0
  Created wheel for pytools: filename=pytools-2020.4.3-py2.py3-none-any.whl size=61374 sha256=1e2efa1b2744179d92638137fc751f7969b02a48b0a4f7e3166b291917b91ea1
  Stored in directory: /root/.cache/pip/wheels/af/c7/81/a22edb90b0b09a880468b2253bb1df8e9f503337ee15432c64
Successfully built pycuda pytools


#3 Desarrollo

In [4]:
#@title Parametros { vertical-output: true, display-mode: "both" }
numeroElementos =  0#@param {type:"number"}

try:
  import pycuda.driver as cuda
  import pycuda.autoinit
  from   pycuda.compiler import SourceModule

  from  datetime import datetime
  import numpy



  # CPU - Defino la memoria del vector en cpu.
  vectorCpu = numpy.random.randn(numeroElementos)
  vectorCpu = vectorCpu.astype(numpy.float32())
  print("vector con numeros:")
  print(vectorCpu)

  # CPU - reservo la memoria GPU.
  vectorGpu = cuda.mem_alloc(vectorCpu.nbytes)

  # GPU - Copio la memoria al GPU.
  cuda.memcpy_htod(vectorGpu,vectorCpu)

  # CPU - ingreso el numero a buscar
  print("Ingrese un numero a buscar:")
  numeroBuscado = input()

  #CPU - Defino la funcion kernel que ejecutará en GPU
  module = SourceModule("""
  __global__ void buscarNumero(int n,float numeroBuscado ,float *pvector1)
  {
      int idx = threadIdx.x + blockIdx.x*blockDim.x;
      
      

      if(idx<n)
      {
        if(pvector1[idx] == numeroBuscado)
        {
            pvector1[0] = 1;
        }
      }
  }

  """)

  kernel = module.get_function("buscarNumero")

  dim_hilo = 256
  dim_bloque = numpy.int( (numeroElementos+dim_hilo-1) / dim_hilo )
  print( "Thread x: ", dim_hilo, ", Bloque x:", dim_bloque )

  kernel( numpy.int32(numeroElementos),numpy.float32(numeroBuscado), vectorGpu, block=( dim_hilo, 1, 1 ),grid=(dim_bloque, 1,1) )

  cuda.memcpy_dtoh(vectorCpu,vectorGpu)

  print("vector con numeros final:")
  print(vectorCpu)

  if vectorCpu[0]== 1:
    print("El numero ha sido encontrado")
  else:
      print("El numero no ha sido encontrado")

except ModuleNotFoundError :
      print("No se compilo el código del armado del ambiente")
      print("Compile el armado del ambiente y vuelva a intentarlo")
except ValueError :
  print("se ingreso un número de elementos menor a 0")
  print("ingrese un número de elementos mayor a 0")
except pycuda.driver.LogicError:
  print("Ha ingresado un número de elementos igual a 0 y CUDA no puede reservar un vector de tamaño 0")
  print("ingrese un número de elementos mayor a 0")
      

vector con numeros:
[]
Ha ingresado un número de elementos igual a 0 y CUDA no puede reservar un vector de tamaño 0
ingrese un número de elementos mayor a 0


#4 Tabla de pasos de ejecución del programa


 Procesador | Funciòn | Detalle
------------|---------|----------
CPU      |  @param                | Lectura del tamaño del vector desde Colab.
CPU      |  import                | Importa los módulos para funcionar.
CPU      |  datetime.now()        | Toma el tiempo actual.
CPU      |  numpy.random.randn( numeroElementos ) | Inicializa el vectorCpu
CPU      |  print()               | informo el vector de números en el que voy a buscar 
**GPU**  |  cuda.mem_alloc()      | Reserva la memoria en GPU.
**GPU**  |  cuda.memcpy_htod()    | Copia las memorias desde el CPU al GPU.
CPU      |  print()               | informo que se debe ingresar un numero a buscar
CPU      |  input()               | ingreso el numeroBuscado por teclado
CPU      |  SourceModule()        | Define el código del kernel donde se realiza la busqueda de un numero, si se encuentra, en la  posicion cero copio un 1 para usarlo como bandera
CPU      |  module.get_function() | Genera la función del kernel GPU
CPU      |  dim_hilo/dim_bloque   | Calcula las dimensiones.
**GPU**  |  kernel()              | Ejecuta el kernel en GPU
CPU      |  cuda.memcpy_dtoh( )   | Copia el resultado desde GPU memoria A a CPU memoria R.
CPU      |  if                    | si la posicion 0 del vectorCpu es igual a 1 significa que lo encontre, de lo contrario no lo encontre
CPU      |  print()               | Informo los resultados.


#5 Conclusiones

#6 Bibliografia

[1] MARKDOWN SYNTAX Colab:[PDF](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/markdown-cheatsheet-online.pdf)

[2] Introducción a Python: [pagina colab](https://github.com/wvaliente/SOA_HPC/blob/main/Documentos/Python_Basico.ipynb)